# Pricing Caps
In this example we will explore how to:
1. Price caps by settings a constant volatility and only using a single curve for discounting and forecasting.
2. Price caps by using two curves, one for forecasting and one for discounting.
3. Price caps by having an input volatility surface.

In [1]:
import QuantLib as ql 
import pandas as pd

We initialize by setting the valuation date and defining our forecasting and discounting curve to be used in pricing.

In [2]:
# Begin by setting the valuation date of which the cap and the floor should be priced at
ql.Settings.instance().evaluationDate = ql.Date(1, 1, 2022)
# Then we initialize the curve we want to use for discounting and forecasting
discount_factors = [1, 0.965, 0.94]  # discount factors
dates = [
    ql.Date(1, 1, 2022),
    ql.Date(1, 1, 2023),
    ql.Date(1, 1, 2024),
]  # maturity dates of the discount factors
day_counter = ql.Actual360()
# Note that we will not strip a curve here, but simply use the discount factors and the dates defined above
# By default QuantLib DiscountCurve will log linearly interpolate between the points.
discount_curve = ql.DiscountCurve(dates, discount_factors, day_counter)
# The curve will note be linked in case we want to update the quotes later on
discount_handle = ql.YieldTermStructureHandle(discount_curve)


The next step involves creating an schedule of dates for which the optionlets of the caps will mature at. 

In [3]:
start_date = ql.Date(1, 1, 2022)
end_date = start_date + ql.Period(12, ql.Months)

# We define the schedule of the cap and floor
schedule = ql.Schedule(
    start_date,                 # Start date of payments
    end_date,                   # End date of payments
    ql.Period(3, ql.Months),    # frequency of payments
    ql.Sweden(),                # Calendar for adjusting for holidays
    ql.ModifiedFollowing,       # Business convention for adjusting for holidays
    ql.ModifiedFollowing,       # Business convention for adjusting for holidays
    ql.DateGeneration.Backward, # Date generation rule for generating the schedule
    False,                      # End of month rule
)

# Create a custom index to track the payments correctly, specifically fixing days.
custom_discount_index= ql.IborIndex(
    "MyIndex",
    ql.Period("3m"),
    0,
    ql.SEKCurrency(),
    ql.Sweden(),
    ql.ModifiedFollowing,
    False,
    ql.Actual360(),
    discount_handle,
)


The last step is to define the pricing engine to use for pricing. We can choose between:

- BlackCapFloorEngine
- BachelierEngine
- AnalyticCapFloorEngine
- TreeCapFloorEngine

In this example we will precede with BlackCapFloorEngine.

In [4]:
# As you have noted by now, the pricing of caps and floors involves creating a floating leg
ibor_leg_discount = ql.IborLeg([1e6], schedule, custom_discount_index)
strike = [0.025]
cap_discount = ql.Cap(ibor_leg_discount, strike)

# The final step is to define a volatility surface, we will use a constant volatility for simplicity
volatility = ql.QuoteHandle(ql.SimpleQuote(0.5))

# Input our discounting and forecasting curve together with our volatility surface to the engine
engine = ql.BlackCapFloorEngine(discount_handle, volatility)
cap_discount.setPricingEngine(engine)
print(cap_discount.NPV())

10831.583434218297


At last we want to show our results of the seperate optionlets.

In [5]:
schedule_dates = schedule.dates()

display_result = lambda _ : pd.DataFrame({
    'price': _.optionletsPrice(),
    'discount_factor': _.optionletsDiscountFactor(),
    'cap_rate': _.capRates(),
    'atm_forward': _.optionletsAtmForward(),
    'std_dev': _.optionletsStdDev(),
    'accrual_start': schedule_dates[:-1],
    'accrual_end' : schedule_dates[1:]
})

display_result(cap_discount)

,price,discount_factor,cap_rate,atm_forward,std_dev,accrual_start,accrual_end
0,2493.450264,0.991254,0.025,0.035290,0.037012,"January 3rd, 2022","April 1st, 2022"
1,2625.359083,0.982488,0.025,0.035296,0.248282,"April 1st, 2022","July 1st, 2022"
2,2846.309041,0.973515,0.025,0.035301,0.352097,"July 1st, 2022","October 3rd, 2022"
3,2866.465047,0.964931,0.025,0.035193,0.434000,"October 3rd, 2022","January 2nd, 2023"


Considering that we have used only one curve as discounting and forecasting, we will now add a second curve for forecasting.

In [6]:
ql.Settings.instance().evaluationDate = ql.Date(1, 1, 2022)

# Similiar to the discount curve we declared previously
forward_rates = [0.04, 0.05, 0.06]
forward_curve = ql.ForwardCurve(dates, forward_rates, day_counter)
forward_handle = ql.YieldTermStructureHandle(forward_curve)

In [7]:
# Create a new index that uses the forward curve for forecasting
custom_forward_index= ql.IborIndex(
    "MyIndex",
    ql.Period("3m"),
    0,
    ql.SEKCurrency(),
    ql.Sweden(),
    ql.ModifiedFollowing,
    False,
    ql.Actual360(),
    forward_handle, # Previously was set to discount_handle
)

In [8]:
# Define a new ibor_leg & cap that uses the new index with forward estimation
ibor_leg_forward = ql.IborLeg([1e6], schedule, custom_forward_index) 
cap_forward = ql.Cap(ibor_leg_forward, strike)

# Input our discounting and forecasting curve together with our volatility surface to the engine
engine_forward = ql.BlackCapFloorEngine(discount_handle, volatility)
cap_forward.setPricingEngine(engine_forward)
print(cap_forward.NPV())

25171.79621353972


In [9]:
schedule_dates = schedule.dates()
display_result(cap_forward)

,price,discount_factor,cap_rate,atm_forward,std_dev,accrual_start,accrual_end
0,6132.002083,0.991254,0.025,0.050307,0.037012,"January 3rd, 2022","April 1st, 2022"
1,6289.142138,0.982488,0.025,0.050317,0.248282,"April 1st, 2022","July 1st, 2022"
2,6465.774497,0.973515,0.025,0.050328,0.352097,"July 1st, 2022","October 3rd, 2022"
3,6284.877495,0.964931,0.025,0.050429,0.434000,"October 3rd, 2022","January 2nd, 2023"


The last step is to consider to have a full volatility surface instead of a constant one.

In [10]:
# Set the settlement day of the volatility surface
settlementDays = 0

# Define the expiries for the volatility surface
expiries = [ql.Period("3M"), ql.Period("6M"), ql.Period("9M"), ql.Period("1Y")]

# Define the strikes for the volatility surface
strikes = [0.010, 0.025, 0.03]

# Define the market quotes for the volatility surface
black_volatility = [[0.98, 0.792, 0.6873], [0.9301, 0.7401, 0.6403], [0.7926, 0.6424, 0.5602], [0.7126, 0.6024, 0.4902]]

# Create a new volatility surface
volatility_surface = ql.CapFloorTermVolSurface(
    settlementDays,
    ql.Sweden(),
    ql.ModifiedFollowing,
    expiries,
    strikes,
    black_volatility,
    day_counter,
)
# Strip the volatility surface for optionlets (caplets) as the input is based on caps
optionlet_surf = ql.OptionletStripper1(volatility_surface, custom_forward_index)

# Call strippedOptionletAdapter to create a handle for the volatility surface
ovs_handle = ql.OptionletVolatilityStructureHandle(
    ql.StrippedOptionletAdapter(optionlet_surf)
)

cap_volatility = ql.Cap(ibor_leg_forward, strike)
# Input our discounting and forecasting curve together with our volatility surface to the engine
engine_volatility = ql.BlackCapFloorEngine(discount_handle, ovs_handle)
cap_volatility.setPricingEngine(engine_volatility)
print(cap_volatility.NPV())

25340.288918668186


In [11]:
display_result(cap_volatility)

,price,discount_factor,cap_rate,atm_forward,std_dev,accrual_start,accrual_end
0,6132.002083,0.991254,0.025,0.050307,0.000000,"January 3rd, 2022","April 1st, 2022"
1,6325.268247,0.982488,0.025,0.050317,0.372127,"April 1st, 2022","July 1st, 2022"
2,6526.008974,0.973515,0.025,0.050328,0.434983,"July 1st, 2022","October 3rd, 2022"
3,6357.009614,0.964931,0.025,0.050429,0.500385,"October 3rd, 2022","January 2nd, 2023"


使用Hagan方法定价二元

In [12]:
calendar = ql.TARGET()
referenceDate = calendar.adjust(ql.Date.todaysDate())
print(referenceDate)
ql.Settings.instance().evaluationDate = referenceDate

volQuote = ql.QuoteHandle(ql.SimpleQuote(0.2))
swaptionVol = ql.ConstantSwaptionVolatility(0, ql.TARGET(), ql.ModifiedFollowing, volQuote, ql.Actual365Fixed())
swvol_handle = ql.SwaptionVolatilityStructureHandle(swaptionVol)

mean_reversion = ql.QuoteHandle(ql.SimpleQuote(0.01))
cms_pricer = ql.LinearTsrPricer(swvol_handle, mean_reversion)

swaptionVolatilityStructure = ql.SwaptionVolatilityStructureHandle(ql.ConstantSwaptionVolatility(referenceDate, calendar, ql.ModifiedFollowing,
                                           volQuote, ql.Actual365Fixed()))
cmspricer = ql.AnalyticHaganPricer(swaptionVolatilityStructure, ql.GFunctionFactory.Standard, mean_reversion)

February 4th, 2025


CMS定价参数

In [13]:
termStructure = ql.RelinkableYieldTermStructureHandle()
termStructure.linkTo(ql.FlatForward(referenceDate, 0.05, ql.Actual365Fixed()))
print(termStructure)
# termStructure.plot()

iborIndex = ql.Euribor6M(termStructure)
print(iborIndex)
swapIndex = ql.EuriborSwapIsdaFixA(ql.Period(10, ql.Years), iborIndex.forwardingTermStructure())
startDate = termStructure.referenceDate() + ql.Period(2, ql.Years)
paymentDate = startDate + ql.Period(6, ql.Months)
endDate = paymentDate
nominal = 1.0
infiniteCap = ql.nullDouble()
infiniteFloor = ql.nullDouble()
gearing = 1.0
spread = 0.0
discount = termStructure.discount(paymentDate)
k = 0.02
# k = 0.07

caplet = ql.CappedFlooredCmsCoupon(paymentDate, nominal,
                                            startDate, endDate,
                                            swapIndex.fixingDays(),
                                            swapIndex,
                                            gearing, spread,
                                            k, infiniteFloor,
                                            startDate, endDate,
                                            iborIndex.dayCounter())

caplet.setPricer(cmspricer)
capletPrice = caplet.price(termStructure)
# apletPrice = caplet.rate()
print(capletPrice)

<QuantLib.QuantLib.RelinkableYieldTermStructureHandle; proxy of <Swig Object of type 'RelinkableHandle< YieldTermStructure > *' at 0x000001FBF41ECB70> >
Euribor6M Actual/360 index
0.008875443635088775


使用black 76进行定价

In [28]:
import QuantLib as ql
import math
import scipy.stats as st

# === 1. 设定市场环境 ===
calendar = ql.TARGET()
referenceDate = ql.Date.todaysDate()
ql.Settings.instance().evaluationDate = referenceDate

# 贴现曲线（5% 平坦曲线）
discountCurve = ql.YieldTermStructureHandle(
    ql.FlatForward(referenceDate, ql.QuoteHandle(ql.SimpleQuote(0.05)), ql.Actual365Fixed())
)

# === 2. 定义 Caplet 参数 ===
fixingDate = referenceDate + ql.Period(2, ql.Years)  # 2 年后
paymentDate = fixingDate + ql.Period(6, ql.Months)  # 付款日期
nominal = 1.0  # 名义本金
strike = 0.02  # 执行利率 2%
volatility = 0.20  # 20% 波动率
meanReversion = 0.005  # Hagan 方法需要的均值回归参数
mean_reversion = ql.QuoteHandle(ql.SimpleQuote(meanReversion))

# 创建 IBOR 指数
iborIndex = ql.Euribor6M(discountCurve)
forwardRate = iborIndex.fixing(fixingDate)



# === 3. 计算 Black 76 Caplet 价格 ===
T = ql.Actual365Fixed().yearFraction(referenceDate, fixingDate)
stdDev = volatility * (T ** 0.5)
d1 = (math.log(forwardRate / strike) + 0.5 * stdDev**2) / stdDev
d2 = d1 - stdDev
Nd1 = st.norm.cdf(d1)  # N(d1)
discountFactor = discountCurve.discount(paymentDate)
caplet_price_black76 = discountFactor * nominal * Nd1 * (forwardRate - strike)


# === 4. 计算 Hagan Approximation Caplet 价格 ===
swapIndex = ql.EuriborSwapIsdaFixA(ql.Period(10, ql.Years), iborIndex.forwardingTermStructure())

caplet = ql.CappedFlooredCmsCoupon(paymentDate, nominal,
                                   fixingDate, paymentDate,
                                   swapIndex.fixingDays(),
                                   swapIndex,
                                   1.0, 0.0,  # Gearing = 1, Spread = 0
                                   strike, ql.nullDouble(),
                                   fixingDate, paymentDate,
                                   iborIndex.dayCounter())

# Hagan 方法需要 Swaption Volatility 结构
swaptionVolHandle = ql.SwaptionVolatilityStructureHandle(
    ql.ConstantSwaptionVolatility(referenceDate, calendar, ql.ModifiedFollowing,
                                  ql.QuoteHandle(ql.SimpleQuote(volatility)),
                                  ql.Actual365Fixed())
)

cmsPricer = ql.AnalyticHaganPricer(swaptionVolHandle, ql.GFunctionFactory.Standard, mean_reversion)  # 均值回归参数
# cms_pricer = ql.LinearTsrPricer(swaptionVolHandle, mean_reversion)
# cmsPricer.setSwaptionVolatility(swaptionVolHandle)
caplet.setPricer(cmsPricer)
caplet_price_hagan = caplet.price(discountCurve)

# === 5. 结果对比 ===
print(f"Caplet 价格（Black 76）: {caplet_price_black76:.6f}")
print(f"Caplet 价格（Hagan Approximation）: {caplet_price_hagan:.6f}")

# === 6. 参数核对 ===
print(f"Forward Rate: {forwardRate:.6f}")
print(f"Discount Factor: {discountFactor:.6f}")
print(f"Swaption Volatility: {swaptionVolHandle.volatility(2.0, 10.0, 0.02)}")
print(f"Forward Swap Rate: {swapIndex.fixing(referenceDate + ql.Period(2, ql.Years))}")

print(f"Caplet Rate (Before Pricer): {caplet.rate()}")

caplet.setPricer(None)  # 重置 Pricer
caplet.setPricer(cmsPricer)  # 重新设置 Hagan Pricer

print(f"Caplet Rate (After Pricer): {caplet.rate()}")

caplet_price_hagan = caplet.price(termStructure)
print(f"Hagan Caplet Price: {caplet_price_hagan}")

print(f"Caplet Rate Before Pricer: {caplet.rate()}")

caplet.setPricer(None)  # 先重置 Pricer
caplet.setPricer(cmsPricer)  # 重新设置 Hagan Pricer

print(f"Caplet Rate After Pricer: {caplet.rate()}")
caplet_price_hagan = caplet.price(termStructure)
print(f"Hagan Caplet Price: {caplet_price_hagan}")

price_direct = cmsPricer.swapletPrice()
print(f"Direct CMS Pricer Price: {price_direct}")

for meanReversion in [0.001, 0.005, 0.01, 0.02]:
    cmsPricer = ql.AnalyticHaganPricer(
        swaptionVolHandle,
        ql.GFunctionFactory.Standard,
        ql.QuoteHandle(ql.SimpleQuote(meanReversion))
    )
    caplet.setPricer(cmsPricer)
    caplet_price_hagan = caplet.price(discountCurve)
    print(f"Mean Reversion: {meanReversion:.3f} -> Caplet 价格（Hagan Approximation）: {caplet_price_hagan:.6f}")


Caplet 价格（Black 76）: 0.026413
Caplet 价格（Hagan Approximation）: 0.008875
Forward Rate: 0.049935
Discount Factor: 0.882678
Swaption Volatility: 0.2
Forward Swap Rate: 0.0513135054864824
Caplet Rate (Before Pricer): 0.01999915138485838
Caplet Rate (After Pricer): 0.01999915138485838
Hagan Caplet Price: 0.008875443635088775
Caplet Rate Before Pricer: 0.01999915138485838
Caplet Rate After Pricer: 0.01999915138485838
Hagan Caplet Price: 0.008875443635088775
Direct CMS Pricer Price: 0.02319593260928888
Mean Reversion: 0.001 -> Caplet 价格（Hagan Approximation）: 0.008875
Mean Reversion: 0.005 -> Caplet 价格（Hagan Approximation）: 0.008875
Mean Reversion: 0.010 -> Caplet 价格（Hagan Approximation）: 0.008875
Mean Reversion: 0.020 -> Caplet 价格（Hagan Approximation）: 0.008875


In [29]:
import QuantLib as ql

# 设置日期
calendar = ql.TARGET()
referenceDate = calendar.adjust(ql.Date.todaysDate())
ql.Settings.instance().evaluationDate = referenceDate

# 创建贴现曲线
discountRate = 0.05
termStructure = ql.YieldTermStructureHandle(
    ql.FlatForward(referenceDate, ql.QuoteHandle(ql.SimpleQuote(discountRate)), ql.Actual365Fixed())
)

# 设置 Black 76 波动率
volatility = 0.2  # 20% Swaption Volatility
volHandle = ql.QuoteHandle(ql.SimpleQuote(volatility))
blackVol = ql.BlackConstantVol(referenceDate, calendar, volHandle, ql.Actual365Fixed())

# 选择 IBOR Index
iborIndex = ql.Euribor6M(termStructure)

# 设定 Caplet 参数
startDate = referenceDate + ql.Period(1, ql.Years)
paymentDate = startDate + ql.Period(6, ql.Months)
strike = 0.02  # Caplet Strike Rate
nominal = 1.0  # 名义本金

# ✅ **正确创建 Caplet**
caplet = ql.Cap(
    [ql.IborCoupon(paymentDate, nominal, startDate, paymentDate, iborIndex.fixingDays(), iborIndex, 1.0, 0.0)],
    [strike]
)

# ✅ **正确绑定 Black76 定价引擎**
black76Engine = ql.BlackCapFloorEngine(termStructure, ql.OptionletVolatilityStructureHandle(blackVol))
caplet.setPricingEngine(black76Engine)

# 计算 Caplet 价格
caplet_price = caplet.NPV()
print(f"Caplet Price (Black 76): {caplet_price}")


TypeError: Wrong number or type of arguments for overloaded function 'new_OptionletVolatilityStructureHandle'.
  Possible C/C++ prototypes are:
    Handle< OptionletVolatilityStructure >::Handle(ext::shared_ptr< OptionletVolatilityStructure > const &)
    Handle< OptionletVolatilityStructure >::Handle()
